In [2]:
import torch
import torch.nn as nn
from torchvision import models
from PIL import Image
import shap
import numpy as np
import os

In [3]:
device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
model_path = r"C:\Users\Ekaansh\OneDrive\Desktop\AB\research\SHAP\best_pneumonia_densenet121.pt"
class_names = ['NORMAL', 'PNEUMONIA']

In [5]:
model = models.densenet121(weights=None)
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, len(class_names))
)
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

C:\Users\Ekaansh\AppData\Local\Temp\ipykernel_5060\832990262.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=de

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [6]:
import shap
import numpy as np
import torch


In [7]:
def preprocess_input(images):
    """images: numpy array of shape (N,H,W,3) with 0-255 pixel values"""
    images = images / 255.0
    images = (images - mean) / std
    # convert to PyTorch CHW
    images = images.transpose(0, 3, 1, 2)
    return torch.tensor(images).float()

In [8]:
def f(x):
    """x is numpy array of shape (N,H,W,3)"""
    x_tensor = preprocess_input(x)
    with torch.no_grad():
        out = model(x_tensor)
        probs = torch.softmax(out, dim=1)
    return probs.cpu().numpy() 

In [9]:
image_paths = [
    r"D:\datasets\chest_xray\val\PNEUMONIA\person1946_bacteria_4875.jpeg",
    r"D:\datasets\chest_xray\val\PNEUMONIA\person1954_bacteria_4886.jpeg",
    r"D:\datasets\chest_xray\val\PNEUMONIA\person1951_bacteria_4882.jpeg"

]
X = np.stack([np.array(Image.open(p).convert("RGB").resize((224, 224))) for p in image_paths])

In [10]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [11]:
def normalize(images):
    """images: numpy array (N,H,W,3)"""
    if images.max() > 1:
        images = images / 255.0
    images = (images - mean) / std
    images = images.transpose(0, 3, 1, 2)  # NHWC -> NCHW
    return torch.tensor(images).float()

In [12]:
background = np.zeros((1, 224, 224, 3), dtype=np.float32)
background = normalize(background)

In [13]:
explainer = shap.GradientExplainer(
    (model, model.features[7]),  # intermediate conv layer
    background,
    local_smoothing=0.5
)

In [14]:
shap_values, indexes = explainer.shap_values(
    normalize(X),
    ranked_outputs=2,
    nsamples=10
)

In [15]:
shap_values = [s.numpy().transpose(0, 2, 3, 1) if torch.is_tensor(s) else s.transpose(0, 2, 3, 1) for s in shap_values]
to_explain_np = X

In [16]:
index_names = np.vectorize(lambda x: class_names[x])(indexes)